# Results visualisation notebook

Use this notebook to visualise results generated from `scripts/run_eval.py`. Set the following variables:

In [ ]:
RESULTS_FILE = 'M:/models/deepinv-experiments/eval_test.json'

In [ ]:
import json
from py_markdown_table.markdown_table import markdown_table
from itertools import zip_longest

def isnotNone(x): return x is not None and x != ""
def filt(lbl): [results[id].get(lbl, "") for id in results.keys()]

with open(RESULTS_FILE, "r") as f:
    results = json.load(f)

In [ ]:
titles, metrics, metrics_init = filt('title'), filt('metrics'), filt('metrics_init')

titles = ["No learning"] + titles
metrics = [metrics_init[0]] + metrics #0 here arbitrary as all the same

In [ ]:
data = [{
    "Method": title,
    "metric1": round(metric[0], 2) if isnotNone(metric[0]) else "",
    "metric2": round(metric[1], 2) if isnotNone(metric[1]) else "",
} for title, metric in zip_longest(titles, metrics)]

print(markdown_table(data).set_params(row_sep="markdown").get_markdown())